Explore text message data and create models to predict if a message is spam or not.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./data/spam.csv")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df['target'] = np.where(df['v1']=='ham', 0, 1)

In [4]:
df = df[['v2', 'target']]
df.head()

,v2,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['v2'], df['target'], random_state=0)

### Question 1
What percentage of the documents in spam_data are spam?

In [6]:
df['target'].mean()*100

13.406317300789663

### Question 2
Fit the training data X_train using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer().fit(X_train)
tokens = vect.get_feature_names()

tokens_list = [(token, len(token)) for token in tokens]
tokens_list_sort = sorted(tokens_list, key=lambda x: x[1], reverse=True)

c:\Users\minhh\Documents\dev\data_analyst\.venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
tokens_list_sort[0]

('com1win150ppmx3age16subscription', 32)

### Question 3
Fit and transform the training data X_train using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing alpha=0.1. Find the area under the curve (AUC) score using the transformed test data.

This function should return the AUC score as a float.

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

vect = CountVectorizer().fit(X_train)
X_train_vectorized = vect.transform(X_train)
model = MultinomialNB(alpha=0.1)
model.fit(X_train_vectorized, y_train)
predictions = model.predict(vect.transform(X_test))

roc_auc_score(y_test, predictions)


0.9712450978727738

### Question 4
Fit and transform the training data X_train using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first.

This function should return a tuple of two series (smallest tf-idfs series, largest tf-idfs series).

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer().fit(X_train)
X_train_vectorized = vect.transform(X_train)
feature_names = np.array(vect.get_feature_names())
tfidf = X_train_vectorized.max(0).toarray()[0]
tfidf.sort()

sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
smallest = pd.Series(tfidf[:20], feature_names[sorted_tfidf_index[:20]])
largest = pd.Series(tfidf[:-21:-1], feature_names[sorted_tfidf_index[:-21:-1]])

### Question 5
Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than 3.

Then fit a multinomial Naive Bayes classifier model with smoothing alpha=0.1 and compute the area under the curve (AUC) score using the transformed test data.

In [21]:
vect = TfidfVectorizer(min_df=3).fit(X_train)
X_train_vectorized = vect.transform(X_train)
model = MultinomialNB(alpha=0.1)
model.fit(X_train_vectorized, y_train)
predict = model.predict(vect.transform(X_test))
roc_auc_score(y_test, predict)

0.9416243654822335

### Question 6
What is the average length of documents (number of characters) for not spam and spam documents?

In [16]:
spam_mean = np.mean(df[df['target']==1]['v2'].apply(len))
spam_mean

138.429718875502

In [18]:
not_spam_mean = np.mean(df[df['target']==0]['v2'].apply(len))
not_spam_mean

71.02176165803108

### Question 7
Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than 5.

Using this document-term matrix and an additional feature, the length of document (number of characters), fit a Support Vector Classification model with regularization C=10000. Then compute the area under the curve (AUC) score using the transformed test data.

In [19]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [22]:
from sklearn.svm import SVC
vect = TfidfVectorizer(min_df=5).fit(X_train)
X_train_vectorized = vect.transform(X_train)
X_train_added = add_feature(X_train_vectorized, X_train.apply(len))
X_test_added = add_feature(vect.transform(X_test), X_test.apply(len))

svm = SVC(C=10000).fit(X_train_added, y_train)

roc_auc_score(y_test, svm.predict(X_test_added))

0.968707026806784

### Question 8
What is the average number of digits per document for not spam and spam documents?

In [23]:
spam_digit = df[df['target']==1]['v2'].str.findall(r'\d').str.len()
no_spam_digit = df[df['target']==0]['v2'].str.findall(r'\d').str.len()

In [24]:
np.mean(spam_digit)

15.759036144578314

In [25]:
np.mean(no_spam_digit)

0.2992746113989637

### Question 9
Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than 5 and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:

- the length of document (number of characters)
- **number of digits per document**
fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

In [27]:
from sklearn.linear_model import LogisticRegression

vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
X_train_vectorized = vect.transform(X_train)
X_test_vectorized = vect.transform(X_test)

X_train_add = add_feature(X_train_vectorized, X_train.apply(len))
X_test_add = add_feature(X_test_vectorized, X_test.apply(len))

X_train_add = add_feature(X_train_vectorized, X_train.str.findall(r'\d').str.len())
X_test_add = add_feature(X_test_vectorized, X_test.str.findall(r'\d').str.len())

model = LogisticRegression(C=100).fit(X_train_add, y_train)

roc_auc_score(y_test, model.predict(X_test_add))

c:\Users\minhh\Documents\dev\data_analyst\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9678709064054463

### Question 10
What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

Hint: Use `\w` and `\W` character classes

In [28]:
spam_non = df[df['target']==1]['v2'].str.findall(r'\W').str.len()
print(np.mean(spam_non))

not_spam_non = df[df['target']==0]['v2'].str.findall(r'\W').str.len()
print(np.mean(not_spam_non))

29.06024096385542
17.375336787564766


### Question 11
Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than 5 and using **character n-grams from n=2 to n=5**.

To tell Count Vectorizer to use character n-grams pass in analyzer='char_wb' which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:

- the length of document (number of characters)
- number of digits per document
- **number of non-word characters (anything other than a letter, digit or underscore.)**
fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients: ['length_of_doc', 'digit_count', 'non_word_char_count']

In [29]:
vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
X_train_vectorized = vect.transform(X_train)
X_test_vectorized = vect.transform(X_test)
X_train_added = add_feature(add_feature(add_feature(X_train_vectorized, X_train.apply(len)), 
                                        X_train.str.findall(r'\d').str.len()),
                            X_train.str.findall(r'\W').str.len())
X_test_added = add_feature(add_feature(add_feature(X_test_vectorized, X_test.apply(len)), 
                                        X_test.str.findall(r'\d').str.len()), 
                            X_test.str.findall(r'\W').str.len())

lm = LogisticRegression(C=100).fit(X_train_added, y_train)
auc = roc_auc_score(y_test, lm.predict(X_test_added))
feature_names = list(vect.get_feature_names())+['length_of_doc', 'digit_count', 'non_word_char_count']
sorted_index = lm.coef_[0].argsort()
smallest_list = np.array(feature_names)[sorted_index[:10]]
largest_list = np.array(feature_names)[sorted_index[:-11:-1]]

c:\Users\minhh\Documents\dev\data_analyst\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\minhh\Documents\dev\data_analyst\.venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [30]:
auc

0.9809793219360643

In [31]:
print('smallest_list: ', smallest_list)
print('largest_list: ', largest_list)

smallest_list:  ['..' '. ' ' i' ' go' '? ' 'ok' 'go' 'ca' 'pe' ' y']
largest_list:  ['digit_count' 'co' 'ne' 'ww' 'xt' ' ch' ' a ' 'ia' 'mob' ' x']
